In [ ]:
# this is just to change the presentation style... Please ignore!
from IPython.display import display, HTML
display(HTML("""<style>
.cm-line { font-size: large !important; }
.dataframe tbody tr th { font-size: large !important; }
.dataframe tbody tr { font-size: large !important; }
.dataframe thead th { font-size: large !important; }
.dataframe thead { font-size: large !important; }
</style>"""))

# Networkx (https://networkx.org/)

- Graphs, DiGraphs, MultiGraphs, Trees
- analysis, search, exploration
- LOTS of algorithms, ready to use

In [ ]:
import networkx as nx
nx.__version__

## Creating a graph

In [ ]:
G = nx.DiGraph()

# Add nodes directly
G.add_node("Stefan")
G.add_node("Alessio")
G.add_edge("Stefan", "Alessio", weight=0)

# New nodes are added automatically
G.add_edge("a", "b", weight=14)
G.add_edge("a", "e", weight=3)
G.add_edge("b", "c", weight=4)
G.add_edge("b", "d", weight=5)
G.add_edge("d", "e", weight=7)
G.add_edge("d", "f", weight=7)
G.add_edge("e", "e", weight=9)

## Explore the Graph

In [ ]:
import matplotlib.pyplot as plt

def draw_graph(G):  # just a small helper
    nx.draw_networkx(G, pos=pos, node_color="red")  # 
    labels = {e: G.edges[e]['weight'] for e in G.edges}
    nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
    return plt.show()
draw_graph(G)

In [ ]:
draw_graph(G)

In [ ]:
print(G.degree["Alessio"])
print(G.in_degree["Alessio"])
print(G.out_degree["Alessio"])

In [ ]:
print(G.degree["b"])
print(G.in_degree["b"])
print(G.out_degree["b"])

## Exploring the Graph

In [ ]:
draw_graph(G)

In [ ]:
print("e's ancestors", nx.ancestors(G, "e"))
print("a's descendants", nx.descendants(G, "a"))

In [ ]:
list(nx.neighbors(G, "b"))

In [ ]:
list(nx.neighbors(G.to_undirected(), "b"))

## Adjacency Matrix

In [ ]:
draw_graph(G)

In [ ]:
print(nx.adjacency_matrix(G))

In [ ]:
print(nx.adjacency_matrix(G).todense())

## And many, many, many, many, many more functions

https://networkx.org/documentation/stable/reference/index.html

# An example with Pandas

- WG22 Dataset from https://people.sc.fsu.edu/~jburkardt/datasets/cities/cities.html
- 22 German cities + distances between them

In [ ]:
import pandas as pd
import itertools

## Import Node Data

In [ ]:
city_df = pd.read_csv("data/wg22_name_xy.txt", header=None, names=["Stadt", "x", "y"])
names = city_df.Stadt.tolist()
city_df

## Import Distances

In [ ]:
with open("data/wg22_dist.txt", "r") as f:
    distance_raw = f.read()

# each block ends with a short line of 12 characters.
rows = [line.split() for line in distance_raw.splitlines()]

distance_matrix = pd.DataFrame(rows, columns=names, index=names, dtype=float)
distance_matrix

## Convert to Dense Representation (Edge-list)

In [ ]:
edges = [(from_, to_, distance_matrix.loc[from_, to_]) 
         for from_, to_ in itertools.product(names, repeat=2)
         if from_ != to_]

df_edges = pd.DataFrame(edges, columns=["From", "To", "Distance"])
df_edges

## Create a subgraph from edgelist

In [ ]:
sub_df = df_edges.sample(frac=0.1, random_state=12345)  # <-- a 10% subset of edges, to make it more interesting

G = nx.from_pandas_edgelist(df=sub_df, source='From', target='To', edge_attr='Distance', create_using=nx.Graph)

In [ ]:
plt.figure(0,figsize=(12,12)) 

# Use geographic node positions
for rowidx, row in city_df.iterrows():
    G.nodes[row.Stadt]["pos"] = (row["x"], row["y"])
pos = nx.get_node_attributes(G, 'pos')


nx.draw(G, pos, with_labels=True, node_color="red")
labels = {e: G.edges[e]['Distance'] for e in G.edges}
nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
plt.show()

# Graph Algorithms

## Calculate Minimum Spanning Tree ? 

In [ ]:
MST = nx.minimum_spanning_tree(G)

In [ ]:
plt.figure(0,figsize=(15,15)) 
nx.draw(MST, pos, with_labels=True, node_color="red")
labels = {e: MST.edges[e]['Distance'] for e in MST.edges}
nx.draw_networkx_edge_labels(MST, pos, edge_labels=labels)
plt.show()

## Traveling Salesman Problem

In [ ]:
node_list = nx.approximation.traveling_salesman_problem(G.to_undirected(), weight="Distance")
print(node_list)

In [ ]:
tsp_G = G.to_directed().edge_subgraph(zip(node_list, node_list[1:]))  # Create a subgraph

plt.figure(0,figsize=(15,15)) 
nx.draw(tsp_G, pos, with_labels=True, node_color="red")
labels = {e: tsp_G.edges[e]['Distance'] for e in tsp_G.edges}
nx.draw_networkx_edge_labels(MST, pos, edge_labels=labels)
plt.show()

## Traverse Graph in DFS fashion

In [ ]:
# list() because we get an iterator
list(nx.dfs_edges(G, "Wuerzburg"))

## Traverse Graph in BFS fashion

In [ ]:
# list() because we get an iterator
list(nx.bfs_edges(G, "Wuerzburg"))

## Calculate Shortest Paths 
(Muenchen => Saarbruecken)

In [ ]:
plt.figure(0,figsize=(12,12)) 
nx.draw(G, pos, with_labels=True, node_color="red")
labels = {e: G.edges[e]['Distance'] for e in G.edges}
nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
plt.show()

In [ ]:
path = nx.shortest_path(G, source="Muenchen",
                        target="Saarbruecken")
print(path, nx.path_weight(G, path, weight="Distance"))

In [ ]:
path = nx.shortest_path(G, source="Muenchen", 
                        target="Saarbruecken",
                        weight="Distance")
print(path, nx.path_weight(G, path, weight="Distance"))

In [ ]:
path = nx.shortest_path(MST, source="Muenchen", 
                        target="Saarbruecken", 
                        weight="Distance")
print(path, nx.path_weight(G, path, weight="Distance"))

# Questions ?